Instructions
Objective:
To determine the variety of date fruit from data describing the colour, length, diameter, and shape.

Data:
Obtained from DATASETS (muratkoklu.com) and used in M. Koklu, R. Kursun, Y.S. Taspinar, and I. Cinar, "Classification of Date Fruits into Genetic Varieties Using Image Analysis," Mathematical Problems in Engineering, Vol.2021, Article ID: 4793293 (2021).

Problem Statement:
In food production it is important to properly label ingredients for both health and business reasons. However, sometimes mistakes are made and there is room for improvement in food labeling practices. A number of different types of dates are grown around the world, and it takes expertise to correctly identify the variety. Your job as a machine learning developer is to create a model that can identify the type of date from external features such as colour, length, diameter and shape factors which have been determined by a computer vision model.

Steps to be completed:
Create a Jupyter notebook and complete the following steps:


In [350]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import confusion_matrix


Data
Load Date_Fruit_Datasets.csv into a pandas dataframe. Print out the header. Use pandas.DataFrame.describe to summarize the data. Using markdown, explain the meaning of the columns (as well as you can with the information available) and make observations about the dataset.


In [351]:
url = "https://raw.githubusercontent.com/Hunteracademic/Neural_network_group_7/master/Date_Fruit_Datasets.csv"
Fruit_data = pd.read_csv(url)

In [352]:
Fruit_data.head()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,KurtosisRR,KurtosisRG,KurtosisRB,EntropyRR,EntropyRG,EntropyRB,ALLdaub4RR,ALLdaub4RG,ALLdaub4RB,Class
0,422163,2378.908,837.8484,645.6693,0.6373,733.1539,0.9947,424428,0.7831,1.2976,...,3.2370,2.9574,4.2287,-5.919126e+10,-50714214400,-39922372608,58.7255,54.9554,47.8400,BERHI
1,338136,2085.144,723.8198,595.2073,0.5690,656.1464,0.9974,339014,0.7795,1.2161,...,2.6228,2.6350,3.1704,-3.423307e+10,-37462601728,-31477794816,50.0259,52.8168,47.8315,BERHI
2,526843,2647.394,940.7379,715.3638,0.6494,819.0222,0.9962,528876,0.7657,1.3150,...,3.7516,3.8611,4.7192,-9.394835e+10,-74738221056,-60311207936,65.4772,59.2860,51.9378,BERHI
3,416063,2351.210,827.9804,645.2988,0.6266,727.8378,0.9948,418255,0.7759,1.2831,...,5.0401,8.6136,8.2618,-3.207431e+10,-32060925952,-29575010304,43.3900,44.1259,41.1882,BERHI
4,347562,2160.354,763.9877,582.8359,0.6465,665.2291,0.9908,350797,0.7569,1.3108,...,2.7016,2.9761,4.4146,-3.998097e+10,-35980042240,-25593278464,52.7743,50.9080,42.6666,BERHI


In [353]:
Fruit_data.describe()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,SkewRB,KurtosisRR,KurtosisRG,KurtosisRB,EntropyRR,EntropyRG,EntropyRB,ALLdaub4RR,ALLdaub4RG,ALLdaub4RB
count,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,...,898.000000,898.000000,898.000000,898.000000,8.980000e+02,8.980000e+02,8.980000e+02,898.000000,898.000000,898.000000
mean,298295.207127,2057.660953,750.811994,495.872785,0.737468,604.577938,0.981840,303845.592428,0.736267,2.131102,...,0.250518,4.247845,5.110894,3.780928,-3.185021e+10,-2.901860e+10,-2.771876e+10,50.082888,48.805681,48.098393
std,107245.205337,410.012459,144.059326,114.268917,0.088727,119.593888,0.018157,108815.656947,0.053745,17.820778,...,0.632918,2.892357,3.745463,2.049831,2.037241e+10,1.712952e+10,1.484137e+10,16.063125,14.125911,10.813862
min,1987.000000,911.828000,336.722700,2.283200,0.344800,50.298400,0.836600,2257.000000,0.512300,1.065300,...,-1.029100,1.708200,1.607600,1.767200,-1.091220e+11,-9.261697e+10,-8.747177e+10,15.191100,20.524700,22.130000
25%,206948.000000,1726.091500,641.068650,404.684375,0.685625,513.317075,0.978825,210022.750000,0.705875,1.373725,...,-0.196950,2.536625,2.508850,2.577275,-4.429444e+10,-3.894638e+10,-3.564534e+10,38.224425,38.654525,39.250725
50%,319833.000000,2196.345450,791.363400,495.054850,0.754700,638.140950,0.987300,327207.000000,0.746950,1.524150,...,0.135550,3.069800,3.127800,3.080700,-2.826156e+10,-2.620990e+10,-2.392928e+10,53.841300,50.337800,49.614100
75%,382573.000000,2389.716575,858.633750,589.031700,0.802150,697.930525,0.991800,388804.000000,0.775850,1.674750,...,0.593950,4.449850,7.320400,4.283125,-1.460482e+10,-1.433105e+10,-1.660367e+10,63.063350,59.573600,56.666675
max,546063.000000,2811.997100,1222.723000,766.453600,1.000000,833.827900,0.997400,552598.000000,0.856200,535.525700,...,3.092300,26.171100,26.736700,32.249500,-1.627316e+08,-5.627727e+08,-4.370435e+08,79.828900,83.064900,74.104600


In [354]:
Fruit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 898 entries, 0 to 897
Data columns (total 35 columns):
AREA             898 non-null int64
PERIMETER        898 non-null float64
MAJOR_AXIS       898 non-null float64
MINOR_AXIS       898 non-null float64
ECCENTRICITY     898 non-null float64
EQDIASQ          898 non-null float64
SOLIDITY         898 non-null float64
CONVEX_AREA      898 non-null int64
EXTENT           898 non-null float64
ASPECT_RATIO     898 non-null float64
ROUNDNESS        898 non-null float64
COMPACTNESS      898 non-null float64
SHAPEFACTOR_1    898 non-null float64
SHAPEFACTOR_2    898 non-null float64
SHAPEFACTOR_3    898 non-null float64
SHAPEFACTOR_4    898 non-null float64
MeanRR           898 non-null float64
MeanRG           898 non-null float64
MeanRB           898 non-null float64
StdDevRR         898 non-null float64
StdDevRG         898 non-null float64
StdDevRB         898 non-null float64
SkewRR           898 non-null float64
SkewRG           898 non-


Use pandas.DataFrame.info to check if the entries are the correct datatype, and if there are any missing values. Use pandas.DataFrame.duplicates to check for duplicate entries. Fix the dataset so that there are no missing values, duplicate rows, or incorrect data types. Use markdown to make observations and explain what you have done.


In [355]:
print(Fruit_data.duplicated().sum())

0


In [356]:
print(Fruit_data.isna().sum())

AREA             0
PERIMETER        0
MAJOR_AXIS       0
MINOR_AXIS       0
ECCENTRICITY     0
EQDIASQ          0
SOLIDITY         0
CONVEX_AREA      0
EXTENT           0
ASPECT_RATIO     0
ROUNDNESS        0
COMPACTNESS      0
SHAPEFACTOR_1    0
SHAPEFACTOR_2    0
SHAPEFACTOR_3    0
SHAPEFACTOR_4    0
MeanRR           0
MeanRG           0
MeanRB           0
StdDevRR         0
StdDevRG         0
StdDevRB         0
SkewRR           0
SkewRG           0
SkewRB           0
KurtosisRR       0
KurtosisRG       0
KurtosisRB       0
EntropyRR        0
EntropyRG        0
EntropyRB        0
ALLdaub4RR       0
ALLdaub4RG       0
ALLdaub4RB       0
Class            0
dtype: int64



Create a bar plot using seaborn.barplot of the number of elements in each category. Use markdown to comment on how well balanced the dataset is.



Move the labels into a separate dataframe and use sklearn.preprocessing.LabelEncoder to convert the string labels into integers. Reshape the labels into a 2d array. Determine which number has been assigned to each type of date and record this information in markdown.


In [357]:
# Creating a separate dataframe for the features and the labels
df_features = Fruit_data.drop(['Class'], axis = 1)
labels = Fruit_data['Class']

# Encoding the labels and converting the strings into integers
encoder = LabelEncoder()
labels_encoded = encoder.fit_transform(labels)

# Reshaping encoded labels into a two-dimensional array
labels_encoded = labels_encoded.reshape(-1, 1)
labels_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))

# Outputting the number assigned to each type
labels_mapping

{'BERHI': 0,
 'DEGLET': 1,
 'DOKOL': 2,
 'IRAQI': 3,
 'ROTANA': 4,
 'SAFAVI': 5,
 'SOGAY': 6}


Use sklearn.preprocessing.MinMaxScaler to scale the features (but not the labels). Split the data into training, testing and validation sets with appropriate proportions.


In [358]:
# Scaling the features 
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df_features)

# Splitting the data into training, validation, and testing sets
x_train, x_valtest, y_train, y_valtest = train_test_split(scaled_features, labels_encoded, train_size = 0.7, random_state = 10)
x_val, x_test, y_val, y_test = train_test_split(x_valtest, y_valtest, train_size = 0.5, random_state = 10)

In [359]:
x_train.shape, x_val.shape, x_test.shape, y_train.shape, y_val.shape, y_test.shape

((628, 34), (135, 34), (135, 34), (628, 1), (135, 1), (135, 1))


Modeling 
Use tf.keras.Sequential to create a fully connected artificial neural network with at least two hidden layers. Choose an activation function for each layer, and make sure the input and output dimensions are appropriate for the data. Print a summary of the model using tf.summary.


In [360]:
# Creating an artificial neural network with two hidden layers 
model = Sequential([
    Dense(units = 64, activation='relu', input_shape=(34,)),  
    Dense(units = 32, activation='relu'),                      
    Dense(units = 7, activation='softmax')                      
])

In [361]:
# A summary of the model
model.summary()

Model: "sequential_101"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_318 (Dense)            (None, 64)                2240      
_________________________________________________________________
dense_319 (Dense)            (None, 32)                2080      
_________________________________________________________________
dense_320 (Dense)            (None, 7)                 231       
Total params: 4,551
Trainable params: 4,551
Non-trainable params: 0
_________________________________________________________________



Compile the model with a choice of optimizer and loss function, and the set the metrics argument equal to ['accuracy'].


In [362]:
# Choosing adam optimizer and binary cross entropy for loss
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Train the model and record the training accuracy. Find the validation accuracy and confusion matrix.


In [363]:
# Training the model on 25 epoch and a batch size of 32
tf.random.set_seed(42)

history = model.fit(
    x_train, y_train,
    epochs=25,
    batch_size=32,
    validation_data=(x_val, y_val),
    verbose=1
)

Train on 628 samples, validate on 135 samples
Epoch 1/25
628/628 [==============================] - 0s 208us/sample - loss: 1.8305 - accuracy: 0.4618 - val_loss: 1.6897 - val_accuracy: 0.6444
Epoch 2/25
628/628 [==============================] - 0s 24us/sample - loss: 1.6348 - accuracy: 0.6035 - val_loss: 1.4820 - val_accuracy: 0.6593
Epoch 3/25
628/628 [==============================] - 0s 25us/sample - loss: 1.4524 - accuracy: 0.6131 - val_loss: 1.2800 - val_accuracy: 0.6593
Epoch 4/25
628/628 [==============================] - 0s 24us/sample - loss: 1.2751 - accuracy: 0.6146 - val_loss: 1.0958 - val_accuracy: 0.6593
Epoch 5/25
628/628 [==============================] - 0s 23us/sample - loss: 1.1229 - accuracy: 0.6162 - val_loss: 0.9589 - val_accuracy: 0.6593
Epoch 6/25
628/628 [==============================] - 0s 26us/sample - loss: 1.0043 - accuracy: 0.6576 - val_loss: 0.8449 - val_accuracy: 0.7037
Epoch 7/25
628/628 [==============================] - 0s 22us/sample - loss: 0.8975

In [364]:
# Training accuracy
model.evaluate(x_train, y_train)

628/628 [==============================] - 0s 13us/sample - loss: 0.3805 - accuracy: 0.8774


[0.3804717654255545, 0.87738854]

In [365]:
# Validation accuracy
model.evaluate(x_val, y_val)

135/135 [==============================] - 0s 22us/sample - loss: 0.3935 - accuracy: 0.8815


[0.39354448450936214, 0.88148147]

In [366]:
# Plotting the confusion matrix
y_test_probs = model.predict(x_test)
y_test_pred = np.argmax(y_test_probs, axis=1)

# True labels
y_test_true = y_test.flatten()

# Confusion matrix
cm = confusion_matrix(y_test_true, y_test_pred)
cm

array([[ 4,  0,  0,  2,  0,  0,  0],
       [ 0,  7,  4,  0,  1,  0,  3],
       [ 0,  1, 36,  0,  0,  0,  0],
       [ 1,  0,  0, 11,  0,  0,  0],
       [ 1,  0,  0,  0, 19,  0,  0],
       [ 0,  0,  0,  0,  0, 32,  0],
       [ 0,  4,  0,  0,  1,  0,  8]], dtype=int64)

An accuracy of 87.5% on the training set is pretty good, and 88.9% on the validation set means our model is performing well. However, we'll try to tune the hyperparameters a bit more to see if the accuracy improves.

Return to the above steps to try at least five different choices of hyperparameters (including dimensions, activation functions, number of layers, optimizer, loss function, etc.). Neatly present the description of each model tried along with the training and validation accuracies, and the confusion matrix.


In [374]:
# Creating another model to check if accuracy improves--adding additional layers and more units
tf.random.set_seed(42)

model_2 = Sequential([
    Dense(units = 256, activation='relu', input_shape=(34,)), 
    Dense(units = 128, activation='relu'),    
    Dense(units = 64, activation='relu'),
    Dense(units = 7, activation='softmax')                  
])
model_2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model_2.fit(
    x_train, y_train,
    epochs=25,
    batch_size=32,
    validation_data=(x_val, y_val),
    verbose=1
)

print("Training accuracy for second model: ", model_2.evaluate(x_train, y_train))
print("Validation accuracy for second model: ", model_2.evaluate(x_val, y_val))

y_test_probs = model_2.predict(x_test)
y_test_pred = np.argmax(y_test_probs, axis=1)
y_test_true = y_test.flatten()
cm = confusion_matrix(y_test_true, y_test_pred)
print(cm)

Train on 628 samples, validate on 135 samples
Epoch 1/25
628/628 [==============================] - 0s 496us/sample - loss: 1.7203 - accuracy: 0.4268 - val_loss: 1.3661 - val_accuracy: 0.6593
Epoch 2/25
628/628 [==============================] - 0s 32us/sample - loss: 1.2267 - accuracy: 0.6258 - val_loss: 0.9184 - val_accuracy: 0.6667
Epoch 3/25
628/628 [==============================] - 0s 32us/sample - loss: 0.8840 - accuracy: 0.6736 - val_loss: 0.6610 - val_accuracy: 0.7704
Epoch 4/25
628/628 [==============================] - 0s 32us/sample - loss: 0.6739 - accuracy: 0.7691 - val_loss: 0.5591 - val_accuracy: 0.8222
Epoch 5/25
628/628 [==============================] - 0s 30us/sample - loss: 0.5642 - accuracy: 0.8121 - val_loss: 0.4984 - val_accuracy: 0.8519
Epoch 6/25
628/628 [==============================] - 0s 28us/sample - loss: 0.5080 - accuracy: 0.8217 - val_loss: 0.4499 - val_accuracy: 0.8370
Epoch 7/25
628/628 [==============================] - 0s 29us/sample - loss: 0.4696

Already with additional layers and more neurons, although the complexity of our model increases, it performs better.

In [368]:
# Creating another model to check if accuracy improves: a different activation function
tf.random.set_seed(42)

model_3 = Sequential([
    Dense(units = 64, activation='tanh', input_shape=(34,)),
    Dense(units = 32, activation='tanh'),    
    Dense(units = 7, activation='softmax')
])
model_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model_3.fit(
    x_train, y_train,
    epochs=25,
    batch_size=32,
    validation_data=(x_val, y_val),
    verbose=1
)
print("Training accuracy for third model: ",model_3.evaluate(x_train, y_train))
print("Validation accuracy for third model: ",model_3.evaluate(x_val, y_val))

y_test_probs = model_3.predict(x_test)
y_test_pred = np.argmax(y_test_probs, axis=1)
y_test_true = y_test.flatten()
cm = confusion_matrix(y_test_true, y_test_pred)
print(cm)

Train on 628 samples, validate on 135 samples
Epoch 1/25
628/628 [==============================] - 0s 201us/sample - loss: 1.7947 - accuracy: 0.3312 - val_loss: 1.5460 - val_accuracy: 0.6593
Epoch 2/25
628/628 [==============================] - 0s 24us/sample - loss: 1.4775 - accuracy: 0.5987 - val_loss: 1.2603 - val_accuracy: 0.6519
Epoch 3/25
628/628 [==============================] - 0s 22us/sample - loss: 1.2460 - accuracy: 0.6162 - val_loss: 1.0547 - val_accuracy: 0.6593
Epoch 4/25
628/628 [==============================] - 0s 24us/sample - loss: 1.0767 - accuracy: 0.6258 - val_loss: 0.9084 - val_accuracy: 0.6815
Epoch 5/25
628/628 [==============================] - 0s 23us/sample - loss: 0.9610 - accuracy: 0.6369 - val_loss: 0.8129 - val_accuracy: 0.6889
Epoch 6/25
628/628 [==============================] - 0s 24us/sample - loss: 0.8738 - accuracy: 0.6927 - val_loss: 0.7376 - val_accuracy: 0.7259
Epoch 7/25
628/628 [==============================] - 0s 22us/sample - loss: 0.7957

In [369]:
# Creating another model to check if accuracy improves: trying out a different loss function
tf.random.set_seed(42)

model_4 = Sequential([
    Dense(units = 64, activation='relu', input_shape=(34,)), 
    Dense(units = 32, activation='relu'),    
    Dense(units = 7, activation='softmax')                    
])
model_4.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model_4.fit(
    x_train, y_train,
    epochs=25,
    batch_size=32,
    validation_data=(x_val, y_val),
    verbose=1
)
print("Training accuracy for fourth model: ",model_4.evaluate(x_train, y_train))
print("Validation accuracy for second model: ",model_4.evaluate(x_val, y_val))

y_test_probs = model_4.predict(x_test)
y_test_pred = np.argmax(y_test_probs, axis=1)
y_test_true = y_test.flatten()
cm = confusion_matrix(y_test_true, y_test_pred)
print(cm)

Train on 628 samples, validate on 135 samples
Epoch 1/25
628/628 [==============================] - 0s 176us/sample - loss: 1.8151 - accuracy: 0.3248 - val_loss: 1.5841 - val_accuracy: 0.6444
Epoch 2/25
628/628 [==============================] - 0s 22us/sample - loss: 1.4592 - accuracy: 0.5987 - val_loss: 1.2037 - val_accuracy: 0.6593
Epoch 3/25
628/628 [==============================] - 0s 23us/sample - loss: 1.1586 - accuracy: 0.6162 - val_loss: 0.9305 - val_accuracy: 0.6667
Epoch 4/25
628/628 [==============================] - 0s 25us/sample - loss: 0.9448 - accuracy: 0.6513 - val_loss: 0.7567 - val_accuracy: 0.6815
Epoch 5/25
628/628 [==============================] - 0s 22us/sample - loss: 0.7991 - accuracy: 0.7038 - val_loss: 0.6821 - val_accuracy: 0.7704
Epoch 6/25
628/628 [==============================] - 0s 22us/sample - loss: 0.7211 - accuracy: 0.7691 - val_loss: 0.5977 - val_accuracy: 0.7852
Epoch 7/25
628/628 [==============================] - 0s 24us/sample - loss: 0.6474

In [370]:
# Creating another model to check if accuracy improves: larger batch size
tf.random.set_seed(42)

model_5 = Sequential([
    Dense(units = 64, activation='relu', input_shape=(34,)),
    Dense(units = 32, activation='relu'),    
    Dense(units = 7, activation='softmax')           
])
model_5.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

for batch_size in [32, 64]:
    model_5.fit(
    x_train, y_train,
    epochs=25,
    batch_size=batch_size,
    validation_data=(x_val, y_val),
    verbose=0
)
    print(f"Training accuracy for fifth model and batch size {batch_size}: ", model_5.evaluate(x_train, y_train))
    print(f"Validation accuracy for fifth model and batch size {batch_size}: ", model_5.evaluate(x_val, y_val))
    y_test_probs = model_5.predict(x_test)
    y_test_pred = np.argmax(y_test_probs, axis=1)
    y_test_true = y_test.flatten()
    cm = confusion_matrix(y_test_true, y_test_pred)
    print(cm)

628/628 [==============================] - 0s 12us/sample - loss: 0.3806 - accuracy: 0.8838
Training accuracy for fifth model and batch size 32:  [0.3806121664442075, 0.88375795]
135/135 [==============================] - 0s 22us/sample - loss: 0.3955 - accuracy: 0.8889
Validation accuracy for fifth model and batch size 32:  [0.39549042516284516, 0.8888889]
[[ 4  0  0  2  0  0  0]
 [ 0  7  4  0  1  0  3]
 [ 0  1 36  0  0  0  0]
 [ 2  0  0 10  0  0  0]
 [ 1  0  0  0 19  0  0]
 [ 0  0  0  0  0 32  0]
 [ 0  2  0  0  1  0 10]]
628/628 [==============================] - 0s 11us/sample - loss: 0.3009 - accuracy: 0.9029
Training accuracy for fifth model and batch size 64:  [0.30087077655610006, 0.90286624]
135/135 [==============================] - 0s 22us/sample - loss: 0.3711 - accuracy: 0.9185
Validation accuracy for fifth model and batch size 64:  [0.37112899555100337, 0.91851854]
[[ 4  0  0  2  0  0  0]
 [ 0  7  4  0  1  0  3]
 [ 0  1 36  0  0  0  0]
 [ 0  0  0 12  0  0  0]
 [ 0  0  0  0

Extra model; increasing epochs on final model.

In [371]:
# Using final model; increase epochs
tf.random.set_seed(42)

model_5 = Sequential([
    Dense(units = 64, activation='relu', input_shape=(34,)),
    Dense(units = 32, activation='relu'),    
    Dense(units = 7, activation='softmax')           
])
model_5.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

for batch_size in [32, 64]:
    model_5.fit(
    x_train, y_train,
    epochs=94,
    batch_size=batch_size,
    validation_data=(x_val, y_val),
    verbose=0
)
    print(f"Training accuracy for fifth model and batch size {batch_size}: ", model_5.evaluate(x_train, y_train))
    print(f"Validation accuracy for fifth model and batch size {batch_size}: ", model_5.evaluate(x_val, y_val))
    y_test_probs = model_5.predict(x_test)
    y_test_pred = np.argmax(y_test_probs, axis=1)
    y_test_true = y_test.flatten()
    cm = confusion_matrix(y_test_true, y_test_pred)
    print(cm)

628/628 [==============================] - 0s 10us/sample - loss: 0.2160 - accuracy: 0.9204
Training accuracy for fifth model and batch size 32:  [0.21596202643433954, 0.92038214]
135/135 [==============================] - 0s 19us/sample - loss: 0.3478 - accuracy: 0.9481
Validation accuracy for fifth model and batch size 32:  [0.34781391046665333, 0.94814813]
[[ 3  0  0  3  0  0  0]
 [ 0  8  3  0  0  0  4]
 [ 0  3 34  0  0  0  0]
 [ 0  0  0 12  0  0  0]
 [ 0  0  0  0 20  0  0]
 [ 0  0  0  0  0 32  0]
 [ 0  4  0  0  0  0  9]]
628/628 [==============================] - 0s 11us/sample - loss: 0.1693 - accuracy: 0.9363
Training accuracy for fifth model and batch size 64:  [0.1693055968565546, 0.93630576]
135/135 [==============================] - 0s 22us/sample - loss: 0.3445 - accuracy: 0.9481
Validation accuracy for fifth model and batch size 64:  [0.3445088236420243, 0.94814813]
[[ 4  0  0  2  0  0  0]
 [ 0  8  3  0  0  0  4]
 [ 0  1 36  0  0  0  0]
 [ 1  0  0 11  0  0  0]
 [ 0  0  0  0


# Conclusion


When examining the validation accuracies across all experiments, it was observed that the best-performing model was the one that increased both the number of layers and the number of units per layer. This increase in model capacity allowed the network to better capture complex patterns in the data, leading to improved generalization on the validation set. Increasing the number of training epochs can also improve performance, as the model has more opportunities to minimize the loss function. However, this approach is more prone to overfitting, especially once the model begins to memorize the training data rather than learn meaningful patterns. Additionally, training for more epochs significantly increases computational time, making it less efficient for identifying the optimal model configuration compared to architectural improvements.

In [382]:
tf.random.set_seed(42)

model_2 = Sequential([
    Dense(units = 256, activation='relu', input_shape=(34,)), 
    Dense(units = 128, activation='relu'),    
    Dense(units = 64, activation='relu'),
    Dense(units = 7, activation='softmax')                  
])
model_2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model_2.fit(
    x_train, y_train,
    epochs=25,
    batch_size=32,
    validation_data=(x_val, y_val),
    verbose=1
)

print("Training accuracy for second model: ", model_2.evaluate(x_train, y_train))
print("Validation accuracy for second model: ", model_2.evaluate(x_val, y_val))

y_test_probs = model_2.predict(x_test)
y_test_pred = np.argmax(y_test_probs, axis=1)
y_test_true = y_test.flatten()
cm = confusion_matrix(y_test_true, y_test_pred)
print(cm)

test_loss, test_accuracy = model_2.evaluate(x_test, y_test, verbose=0)
print("Testing accuracy for second model:", test_accuracy)

Train on 628 samples, validate on 135 samples
Epoch 1/25
628/628 [==============================] - 0s 232us/sample - loss: 1.7203 - accuracy: 0.4268 - val_loss: 1.3661 - val_accuracy: 0.6593
Epoch 2/25
628/628 [==============================] - 0s 30us/sample - loss: 1.2267 - accuracy: 0.6258 - val_loss: 0.9184 - val_accuracy: 0.6667
Epoch 3/25
628/628 [==============================] - 0s 30us/sample - loss: 0.8840 - accuracy: 0.6736 - val_loss: 0.6610 - val_accuracy: 0.7704
Epoch 4/25
628/628 [==============================] - 0s 31us/sample - loss: 0.6739 - accuracy: 0.7691 - val_loss: 0.5591 - val_accuracy: 0.8222
Epoch 5/25
628/628 [==============================] - 0s 30us/sample - loss: 0.5642 - accuracy: 0.8121 - val_loss: 0.4984 - val_accuracy: 0.8519
Epoch 6/25
628/628 [==============================] - 0s 29us/sample - loss: 0.5080 - accuracy: 0.8217 - val_loss: 0.4499 - val_accuracy: 0.8370
Epoch 7/25
628/628 [==============================] - 0s 29us/sample - loss: 0.4696


The model performs well for the intended use case, achieving a testing accuracy of 88.1%, which indicates strong generalization to unseen data. While performance decreases slightly from training and validation accuracy, this behavior is expected and suggests that the model does not significantly overfit the training data.

The confusion matrix shows that the model correctly classifies the majority of samples across all seven classes, with particularly strong performance in several classes that exhibit near-perfect prediction accuracy. Some misclassifications occur between a small subset of classes, likely due to overlapping feature patterns, which is common in multi-class classification problems.

Overall, the model provides reliable predictions for the use case and is suitable for practical application. Further improvements could be achieved by refining feature selection or increasing data representation for classes that are more frequently confused.